In [1]:
# py37NLP
# Check GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4257559412053958435
]


2022-06-01 07:40:35.611342: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# conda install -c conda-forge ipywidgetsy

import re
from nltk.tokenize import sent_tokenize
from happytransformer import HappyGeneration
from happytransformer import GENSettings

happy_gen = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-2.7B")

/Users/kimkwangil/opt/anaconda3/envs/py37NLP/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
06/01/2022 07:42:09 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [3]:
def selectFromDict(options, name):

    index = 0
    indexValidList = []
    print('Select a ' + name + ':')
    for optionName in options:
        index = index + 1
        indexValidList.extend([options[optionName]])
        print(str(index) + ') ' + optionName)
    inputValid = False
    while not inputValid:
        inputRaw = input(name + ': ')
        inputNo = int(inputRaw) - 1
        if inputNo > -1 and inputNo < len(indexValidList):
            selected = indexValidList[inputNo]
            print('Selected ' +  name + ': ' + selected)
            inputValid = True
            break
        else:
            print('Please select a valid ' + name + ' number')

    return selected

In [4]:
# Gramma-correction
from happytransformer import  HappyTextToText
grammaCorrection = HappyTextToText("T5", "vennify/t5-base-grammar-correction")
from happytransformer import TTSettings

beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=100)

def grmmaCorrection(input_sentence):
    output_text= grammaCorrection.generate_text(input_sentence, args=beam_settings)
    return output_text

06/01/2022 07:42:20 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [5]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [6]:
# 마지막 생성문장을 완결 문장으로 변환 기능 추가
def GenText(input_txt):
    tmp = [0.90, 0.90, 0.90, 0.90, 0.90, 0.90]
    resp = []
    resp_ = []
    resp__ = []
    for temp in tmp:
        top_k_sampling_settings = GENSettings(do_sample=True, 
                                                top_k=200, 
                                                temperature=temp,  
                                                max_length=100, 
                                                no_repeat_ngram_size=2)

        result_top_k_sampling = happy_gen.generate_text(input_txt, args=top_k_sampling_settings)

        #전처리 필요함 GenerationResult(text=' 
        result_top_k_sampling_re = result_top_k_sampling.text
        result_top_k_sampling_re.strip("'")
        result_top_k_sampling_re.lstrip("GenerationResult(text=' ")
        result_top_k_sampling_re = re.sub(r"\n", "", result_top_k_sampling_re)


        
        
        # 생성된 글을 sentence tokenize 
        sent_token_re = sent_tokenize(result_top_k_sampling_re)
        
        # 마지막 문장 가져오기
        inp_sent = str(sent_token_re[-1])
        # 완결된 문장으로 수정하기
        fixed_sent = grmmaCorrection(inp_sent)
        
        #전처리 필요함 TextToTextResult(text=' 
        fixed_sentg_re = fixed_sent.text
        fixed_sentg_re.strip("'")
        fixed_sentg_re.lstrip("TextToTextResult(text=' ")
        fixed_sentg_re = re.sub(r"\n", "", result_top_k_sampling_re)
        
        
        
        fixed_sent_re_ = str(fixed_sentg_re)
        
        # 마지막 요소 제거 후
        sent_token_re.pop()
        # 완결문장 추가
        sent_token_re.append(fixed_sent_re_)
        
        
        for i in sent_token_re:
            resp_.append(i)
            
        result = listToString(resp_)
        resp__.append(result)
        
    return resp__

In [7]:
# # test
# input_txt = """I became fascinated with Spanish and aware of its similarities with """
# GenText(input_txt)

In [8]:
from pprint import pprint

def GenText(input_txt):
    tmp = [0.90, 0.90, 0.90, 0.90, 0.95, 0.95]
    resp = []
    resp_ = []
    resp__ = []
    for temp in tmp:
        top_k_sampling_settings = GENSettings(do_sample=True, 
                                                top_k=200, 
                                                temperature=temp,  
                                                max_length=100, 
                                                no_repeat_ngram_size=2)

        result_top_k_sampling = happy_gen.generate_text(input_txt, args=top_k_sampling_settings)

        #전처리 필요함 GenerationResult(text=' 
        result_top_k_sampling_re = result_top_k_sampling.text
        result_top_k_sampling_re.strip("'")
        result_top_k_sampling_re.lstrip("GenerationResult(text=' ")
        result_top_k_sampling_re = re.sub(r"\n", "", result_top_k_sampling_re)


        # 생성된 글을 sentence tokenize 
        sent_token_re = sent_tokenize(result_top_k_sampling_re)
        
        # 마지막 문장 가져오기
        inp_sent = str(sent_token_re[-1])
        # 완결된 문장으로 수정하기
        fixed_sent = grmmaCorrection(inp_sent)
        
        #전처리 필요함 TextToTextResult(text=' 
        fixed_sentg_re = fixed_sent.text
        fixed_sentg_re.strip("'")
        fixed_sentg_re.lstrip("TextToTextResult(text=' ")
        fixed_sentg_re = re.sub(r"\n", "", result_top_k_sampling_re)
        
        fixed_sent_re = str(fixed_sent)
        
        # 마지막 요소 제거 후
        sent_token_re.pop()
        # 완결문장 추가
        sent_token_re.append(fixed_sent_re)
        
        
        for i in sent_token_re:
            resp_.append(i)
            
        result = listToString(resp_)
        resp__.append(result)
        
        # 결과물 오직 중간 확인하기 위한 코드임
        for i in resp__:
            print("Generated sentence : ", i)
        
    return resp__
        
        



In [9]:
def GenText_RisingAction_for_Step04(input_txt):
    # tmp is temprature
    tmp = [0.90, 0.90, 0.90, 0.90, 0.95, 0.95]
    insert_txt = ["Then, an unexpected thing happened.", "Then, a sad thing happened.", "Then, a wonderful thing happened.", "", "Then, an interesting thing happened.", "Then, the situation got bad."]
    resp = []
    resp_ = []
    resp__ = []
    for i in range(len(tmp)):
        top_k_sampling_settings = GENSettings(do_sample=True, 
                                                top_k=200, 
                                                temperature=tmp[i],  
                                                max_length=100, 
                                                no_repeat_ngram_size=2)
        print("-------")
        
        intText = str(insert_txt[i])
        input_txt_re = str(input_txt) + intText
              
        print("input text:" , input_txt_re)
        result_top_k_sampling = happy_gen.generate_text(input_txt_re, args=top_k_sampling_settings)

        #전처리 필요함 GenerationResult(text=' 
        result_top_k_sampling_re = result_top_k_sampling.text
        result_top_k_sampling_re.strip("'")
        result_top_k_sampling_re.lstrip("GenerationResult(text=' ")
        result_top_k_sampling_re = re.sub(r"\n", "", result_top_k_sampling_re)

        # 생성된 글을 sentence tokenize 
        sent_token_re = sent_tokenize(result_top_k_sampling_re)
        
        # 마지막 문장 가져오기
        inp_sent = str(sent_token_re[-1])
        # 완결된 문장으로 수정하기
        fixed_sent = grmmaCorrection(inp_sent)
        
        #전처리 필요함 TextToTextResult(text=' 
        fixed_sentg_re = fixed_sent.text
        fixed_sentg_re.strip("'")
        fixed_sentg_re.lstrip("TextToTextResult(text=' ")
        fixed_sentg_re.lstrip("TextToTextResult(text=")
        fixed_sentg_re = re.sub(r"\n", "", result_top_k_sampling_re)
        
        fixed_sent_re = str(fixed_sent)
        
        # 마지막 요소 제거 후
        sent_token_re.pop()
        # 완결문장 추가
        sent_token_re.append(fixed_sent_re)
        
        
        for i in sent_token_re:
            resp_.append(i)
            
        result = listToString(resp_)
        resp__.append(result)
        
        # 결과물 오직 중간 확인하기 위한 코드임
        for i in resp__:
            print("Generated sentence : ", i)
        
    return resp__

In [10]:
# insert_text = """Volunteering at a cancer treatment center has helped me discover my path. When I see patients trapped in not only the hospital but also a moment in time by their diseases, I"""

# GenText_RisingAction_for_Step04(insert_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------
input text: Volunteering at a cancer treatment center has helped me discover my path. When I see patients trapped in not only the hospital but also a moment in time by their diseases, IThen, an unexpected thing happened.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated sentence :  Cancer, like drugs, seems to me to have a way of bringing out the best in the people who fight it. After coming out on the other side of my struggle, it looked like I had been reborn. Yes, cancer got me. But it also got better. I came out of it with a new energy that I will use to help others, even as I continue my own fight. TextToTextResult(text='"—Dorothee Puff, 49, Tuscaloosa, AL, volunteer.')
-------
input text: Volunteering at a cancer treatment center has helped me discover my path. When I see patients trapped in not only the hospital but also a moment in time by their diseases, IThen, a sad thing happened.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated sentence :  Cancer, like drugs, seems to me to have a way of bringing out the best in the people who fight it. After coming out on the other side of my struggle, it looked like I had been reborn. Yes, cancer got me. But it also got better. I came out of it with a new energy that I will use to help others, even as I continue my own fight. TextToTextResult(text='"—Dorothee Puff, 49, Tuscaloosa, AL, volunteer.')
Generated sentence :  Cancer, like drugs, seems to me to have a way of bringing out the best in the people who fight it. After coming out on the other side of my struggle, it looked like I had been reborn. Yes, cancer got me. But it also got better. I came out of it with a new energy that I will use to help others, even as I continue my own fight. TextToTextResult(text='"—Dorothee Puff, 49, Tuscaloosa, AL, volunteer.')  I didn't want to think about "cancer" or "fibromyalgia" and I never shared my journey with anyone. But then I realized after all those years of strugglin

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated sentence :  Cancer, like drugs, seems to me to have a way of bringing out the best in the people who fight it. After coming out on the other side of my struggle, it looked like I had been reborn. Yes, cancer got me. But it also got better. I came out of it with a new energy that I will use to help others, even as I continue my own fight. TextToTextResult(text='"—Dorothee Puff, 49, Tuscaloosa, AL, volunteer.')
Generated sentence :  Cancer, like drugs, seems to me to have a way of bringing out the best in the people who fight it. After coming out on the other side of my struggle, it looked like I had been reborn. Yes, cancer got me. But it also got better. I came out of it with a new energy that I will use to help others, even as I continue my own fight. TextToTextResult(text='"—Dorothee Puff, 49, Tuscaloosa, AL, volunteer.')  I didn't want to think about "cancer" or "fibromyalgia" and I never shared my journey with anyone. But then I realized after all those years of strugglin

['Cancer, like drugs, seems to me to have a way of bringing out the best in the people who fight it. After coming out on the other side of my struggle, it looked like I had been reborn. Yes, cancer got me. But it also got better. I came out of it with a new energy that I will use to help others, even as I continue my own fight. TextToTextResult(text=\'"—Dorothee Puff, 49, Tuscaloosa, AL, volunteer.\')',
 'Cancer, like drugs, seems to me to have a way of bringing out the best in the people who fight it. After coming out on the other side of my struggle, it looked like I had been reborn. Yes, cancer got me. But it also got better. I came out of it with a new energy that I will use to help others, even as I continue my own fight. TextToTextResult(text=\'"—Dorothee Puff, 49, Tuscaloosa, AL, volunteer.\')  I didn\'t want to think about "cancer" or "fibromyalgia" and I never shared my journey with anyone. But then I realized after all those years of struggling with my symptoms that I wasn\'t

In [11]:
def GenText_Climax_for_Step05(input_txt):
    # tmp is temprature
    tmp = [0.90, 0.90, 0.90, 0.90, 0.95, 0.95]
    insert_txt = ["Then, things got out of control.", "Then, things got more critical.", "Then, things got even more interesting.", "Then, the situation peaked.", "", ""]
    resp = []
    resp_ = []
    resp__ = []
    for i in range(len(tmp)):
        top_k_sampling_settings = GENSettings(do_sample=True, 
                                                top_k=200, 
                                                temperature=tmp[i],  
                                                max_length=100, 
                                                no_repeat_ngram_size=2)
        
        print("insert_txt[i] :", str(insert_txt[i]))
        
        input_txt_re = input_txt + str(insert_txt[i])
        print("input text:" , input_txt_re)
        result_top_k_sampling = happy_gen.generate_text(input_txt_re, args=top_k_sampling_settings)

        #전처리 필요함 GenerationResult(text=' 
        result_top_k_sampling_re = result_top_k_sampling.text
        result_top_k_sampling_re.strip("'")
        result_top_k_sampling_re.lstrip("GenerationResult(text=' ")
        result_top_k_sampling_re = re.sub(r"\n", "", result_top_k_sampling_re)

        # 생성된 글을 sentence tokenize 
        sent_token_re = sent_tokenize(result_top_k_sampling_re)
        
        # 마지막 문장 가져오기
        inp_sent = str(sent_token_re[-1])
        # 완결된 문장으로 수정하기
        fixed_sent = grmmaCorrection(inp_sent)
        
        #전처리 필요함 TextToTextResult(text=' 
        fixed_sentg_re = fixed_sent.text
        fixed_sentg_re.strip("'")
        fixed_sentg_re.lstrip("TextToTextResult(text=' ")
        fixed_sentg_re.lstrip("TextToTextResult(text=")
        fixed_sentg_re = re.sub(r"\n", "", result_top_k_sampling_re)
        
        fixed_sent_re = str(fixed_sent)
        
        # 마지막 요소 제거 후
        sent_token_re.pop()
        # 완결문장 추가
        sent_token_re.append(fixed_sent_re)
        
        
        for i in sent_token_re:
            resp_.append(i)
            
        result = listToString(resp_)
        resp__.append(result)
        
        # 결과물 오직 중간 확인하기 위한 코드임
        for i in resp__:
            print("Generated sentence : ", i)
        
    return resp__

In [12]:
def GenText_Resolution_for_Step06(input_txt, selectec_topic):
    # tmp is temprature
    tmp = [0.90, 0.90, 0.90, 0.90, 0.95, 0.95]
                  
    pre_sent_idea01 = "The lesson I learned about my " + selectec_topic + "is the following."
    pre_sent_idea03 = "The lesson I learned about my " + selectec_topic + "is the following."
                  
    insert_txt = [pre_sent_idea01, "In conclusion, the lesson I've learned is this.", pre_sent_idea03, "In conclusion, it matters to be for the following reasons.", "So what did I learned from all this?", ""]
    resp = []
    resp_ = []
    resp__ = []
    for i in range(len(tmp)):
        top_k_sampling_settings = GENSettings(do_sample=True, 
                                                top_k=200, 
                                                temperature=tmp[i],  
                                                max_length=100, 
                                                no_repeat_ngram_size=2)

        intTxt = str(insert_txt[i])
        input_txt_re = input_txt + intTxt
        print("input text of input relolution:" , input_txt_re)
        result_top_k_sampling = happy_gen.generate_text(input_txt_re, args=top_k_sampling_settings)

        #전처리 필요함 GenerationResult(text=' 
        result_top_k_sampling_re = result_top_k_sampling.text
        result_top_k_sampling_re.strip("'")
        result_top_k_sampling_re.lstrip("GenerationResult(text=' ")
        result_top_k_sampling_re = re.sub(r"\n", "", result_top_k_sampling_re)


        # 생성된 글을 sentence tokenize 
        sent_token_re = sent_tokenize(result_top_k_sampling_re)
        
        # 마지막 문장 가져오기
        inp_sent = str(sent_token_re[-1])
        # 완결된 문장으로 수정하기
        fixed_sent = grmmaCorrection(inp_sent)
        
        #전처리 필요함 TextToTextResult(text=' 
        fixed_sentg_re = fixed_sent.text
        fixed_sentg_re.strip("'")
        fixed_sentg_re.lstrip("TextToTextResult(text=' ")
        fixed_sentg_re.lstrip("TextToTextResult(text=")
        fixed_sentg_re = re.sub(r"\n", "", result_top_k_sampling_re)
        
        fixed_sent_re = str(fixed_sent)
        
        # 마지막 요소 제거 후
        sent_token_re.pop()
        # 완결문장 추가
        sent_token_re.append(fixed_sent_re)
        
        
        for i in sent_token_re:
            resp_.append(i)
            
        result = listToString(resp_)
        resp__.append(result)
        
        # 결과물 오직 중간 확인하기 위한 코드임
        for i in resp__:
            print("Generated sentence : ", i)
        
    return resp__

In [13]:


# GenText_Climax_for_Step05("""I became desperately devoted to my education because I saw knowledge as the key to freeing myself from the chains of ignorance. """)



In [14]:
def ExceptionInput(input_txt):
    if (input_txt.isdigit()):
        print("An incorrect input was detected. Try again.")

    elif (len(input_txt) <= 8):
        print("The sentence is too short. Try again.")

    elif (input_txt == None):
        print("Try again.")
    else:
        pass

    return input_txt

In [15]:
def Step_01():
    print("Step #1")
    print("I am using Common App OR Coalition App")
    questions = {}
    questions['CommonApp'] = 'CommonApp'
    questions['CoalitionApp'] = 'CoalitionApp'
    
    try:
        # Let user select a script
        selected_app_input = selectFromDict(questions, 'Question')
        print("selected_app_input[0] :" , selected_app_input[0])
    except:
        selected_app_input = selectFromDict(questions, 'Question')
        print("selected_app_input[0] :" , selected_app_input)
        ExceptionInput(selected_prompt_input[0])
        Step_01()
    return selected_app_input

In [16]:
#Step_01()

In [17]:
def Step_02():
    print("Step #2")
    print("Select prompt")
    questions = {}
    questions['prompt01'] = 'Some students have a background, identity, interest, or talent that is so meaningful they believe their application would be incomplete without it. If this sounds like you, then please share your story.'
    questions['prompt02'] = 'The lessons we take from obstacles we encounter can be fundamental to later success. Recount a time when you faced a challenge, setback, or failure. How did it affect you, and what did you learn from the experience?'
    questions['prompt03'] = 'Reflect on a time when you questioned or challenged a belief or idea. What prompted your thinking? What was the outcome?'
    questions['prompt04'] = 'Reflect on something that someone has done for you that has made you happy or thankful in a surprising way. How has this gratitude affected or motivated you?'
    questions['prompt05'] = 'Discuss an accomplishment, event, or realization that sparked a period of personal growth and a new understanding of yourself or others.'
    questions['prompt06'] = 'Describe a topic, idea, or concept you find so engaging that it makes you lose all track of time. Why does it captivate you? What or who do you turn to when you want to learn more?'
    questions['prompt07'] = "Share an essay on any topic of your choice. It can be one you've already written, one that responds to a different prompt, or one of your own design."
    
    
    try:
        # Let user select a script
        selected_prompt_input = selectFromDict(questions, 'question')
    except:
        selected_prompt_input = selectFromDict(questions, 'question')
        ExceptionInput(selected_prompt_input)
        Step_02()
        
    if selected_prompt_input == """Some students have a background, identity, interest, or talent that is so meaningful they believe their application would be incomplete without it. If this sounds like you, then please share your story.""":
        background = """1)Background : What's unique about your family, education history, socioeconomic status, ethnicity, hometown, etc. You can write about your background and make sure to tell a story about how it affected your personal growth, values, beliefs, or perspectives for the future."""
        identity = """2)Identity : Your identity can be associated with your culture, race, nationality, religion, gender, disability, etc. Although your ethnic identity is determined at birth, many identities are formed by your experiences in the society and the communities you are a part of. How did your identity come about? What does it mean to you? Express why your unique identity will be a valuable addition to the college's diversity."""
        interest = """3)Interest : What is an interest that is "so meaningful" to you? Since this prompt intends to understand how unique you are, the interest you mention should be something you are passionate about or somewhat unordinary. Can you talk about an interest that defines who you are without turning it into a typical sport, music, art, or academic essay? Rather than describing your interest only, include how the lessons from this interest affected your growth as a person."""
        talent = """4)Talent : Talent could be a hard or soft skill. Making cool things, excelling in certain areas, or managing big projects are definitely solid hard skills. Also, making new friends, leading people, or empathizing with and helping others can be excellent soft skills. Avoid "bragging" or "humble bragging " when writing about your talent." Don't simply focus on how great you are, but mention the challenges you overcame or how your strength benefitted others."""
        
        topics_result = "Key Topics :" + background + identity + interest + talent
        
        # Select topic
        topics = {}
        topics['background'] = "background"
        topics['identity'] = "identity"
        topics['interest'] = "interest"
        topics['talent'] = "talent"
        
        selected_topic = selectFromDict(topics, 'topic')
        
        if selected_topic == 'background':
            desc = background
        elif selected_topic == 'identity':
            desc = identity
        elif selected_topic == 'interest':
            desc = interest
        else: #selected_topic == 'talent':
            desc = talent
    
    elif selected_prompt_input == """The lessons we take from obstacles we encounter can be fundamental to later success. Recount a time when you faced a challenge, setback, or failure. How did it affect you, and what did you learn from the experience?""":
        challenge = """1)Challenge : The prompt is not only interested in learning about the obstacle you've faced, but you should explain how you overcame such difficulty. Challenges, setbacks, and failures may seem similar in nature. However, an intriguing challenge should be a project, situation, or task that requires courage to go beyond your limitations to increase your capacity as a person."""
        setback = """2)Setback : In general, a setback is a difficulty that you encounter regardless of your will. Usually, injuries, illness, issues with mental health, parents' divorce or bankruptcy, fights, and more would be considered setbacks in life. However, instead of focusing on the difficulty, you should focus on conveying the transition in your mental state, showing how you found a way to stay resilient and optimistic, regardless of your situation."""
        failure = """3)Failure : People are afraid of failure. However, a failure commonly requires one's initiative or risk-taking initially. Because colleges want students willing to take risks to innovate things in the world, writing about your failure can be a powerful topic. Your failures may or may not lead to eventual success in your essay. However, make sure that the lesson you've learned from your failure is insightful because it shows your potential to change the world for the better."""
    
        topics_result = "Key Topics :" + challenge + setback + failure
        
        # Select topic
        topics = {}
        topics['challenge'] = "challenge"
        topics['setback'] = "setback"
        topics['failure'] = "failure"
        
        selected_topic = selectFromDict(topics, 'topic')
        
        if selected_topic == 'challenge':
            desc = challenge
        elif selected_topic == 'setback':
            desc = setback
        else: #selected_topic == 'failure':
            desc = failure
        
    elif selected_prompt_input == """Reflect on a time when you questioned or challenged a belief or idea. What prompted your thinking? What was the outcome?""":
        question = """1)Question : Questioning a belief or idea can involve disagreements which can be discomforting at times. Some beliefs or ideas you oppose are blatantly wrong, such as discriminating against someone. However, for the most part, people simply hold varying perspectives. So rather than showing that you are correct or more logical than others, try to demonstrate how you've been attempting to communicate your thoughts with others in a mature manner."""
        challenge = """2)Challenge : Challenging a belief or idea usually takes place when there is a need for change. Questioning can be an internal struggle, but challenging a person, system, or belief is accompanied by your action. You should explain why making such a change in your community (your family, school, town, etc.) matters to you and what tangible or intangible positive changes took place."""
    
        topics_result = "Key Topics :" + question + challenge
        
        # Select topic
        topics = {}
        topics['question'] = "question"
        topics['challenge'] = "challenge"

        selected_topic = selectFromDict(topics, 'topic')
        
        if selected_topic == 'question':
            desc = question
        else: #selected_topic == 'challenge':
            desc = challenge
        
        
    elif selected_prompt_input == """Reflect on something that someone has done for you that has made you happy or thankful in a surprising way. How has this gratitude affected or motivated you?""":
        gratitude = """1)Qratitude : Feeling grateful toward others is an essential quality of a humble and positive person. It's even better if such a feeling of gratitude leads you to empathize with others in need and fuels your action. If you are dedicated to volunteer activities and helping others, this is a perfect place to show what motivated you to give back to your community."""
        happiness = """2)Happiness : The prompt intends to learn how you don't take things for granted and know how to find happiness from the kindness of others. This is relevant to having a positive attitude; an important quality sought after by colleges. You can show your optimism towards making our society a better place and how you are contributing to your daily life."""
    
        topics_result = "Key Topics :" + gratitude + happiness
        
        # Select topic
        topics = {}
        topics['gratitude'] = "gratitude"
        topics['happiness'] = "happiness"

        selected_topic = selectFromDict(topics, 'topic')
        
        if selected_topic == 'gratitude':
            desc = gratitude
        else: #selected_topic == 'happiness':
            desc = happiness
        
        
    elif selected_prompt_input == """Discuss an accomplishment, event, or realization that sparked a period of personal growth and a new understanding of yourself or others""":
        accomplishmenht = """1)Accomplishmenht : You may think of an accomplishment as something extraordinary such as winning a big award or leading your team to victory. However, the main focus of this essay should be your personal growth, not boasting about your credentials. Instead of struggling to find a big achievement, you can start with a moment that led you to be more mature as a person, then work backward to see what caused it."""
        event = """2)Event : In many cases, personal growth is triggered by a change in your perspective. You should think of an event that humbled you and led you to become a more thoughtful and considerate person. Such an event can be a failure or an initiative you had a sense of conviction. Maybe you had a mentor, a friend, or even a stranger who taught you to see things differently. You may have felt proud or embarrassed towards the outcome of the event. Either way, your story should demonstrate a lesson and how it led you to become more mature as a person."""
        realization = """3)Realization = A realization can come from everywhere. It can be from your everyday life, reading a book, or watching a movie. Since the prompt mentions an accomplishment or event, including your own experience in the story would be more relevant than writing a review paper on a book or Youtube clip. Try to think about incidents where you've experienced hardship, challenge, or even failure and how this led to a new understanding. """
    
        topics_result = "Key Topics :" + accomplishmenht + event + realization
        
        # Select topic
        topics = {}
        topics['accomplishmenht'] = "accomplishmenht"
        topics['event'] = "event"
        topics['realization'] = "realization"
        
        selected_topic = selectFromDict(topics, 'topic')
        
        if selected_topic == 'accomplishmenht':
            desc = accomplishmenht
        elif selected_topic == 'event':
            desc = event
        else: #selected_topic == 'realization':
            desc = realization
    
    elif selected_prompt_input == """Describe a topic, idea, or concept you find so engaging that it makes you lose all track of time. Why does it captivate you? What or who do you turn to when you want to learn more?""":
        topic = """1)Topic : An engaging topic varies from person to person. It can be something intellectual, social, cultural, and more. In addition, it should be a topic you are passionate about, one that you dedicate much time and effort to explore. However, please don't turn your essay into a research paper on the topic. Instead, try to explain why this topic means so much in our lives and how it has affected you in terms of your values or attitude towards life."""
        idea = """2)Idea = An engaging idea can be something academic, social, or simply creative. However, the prompt is not asking you to describe yourself as a daydreamer or a mere observer. It should be an idea meaningful enough to influence your mindset and behavior. For example, it can be something intellectual or your thoughts on how to make a better community. Furthermore, it would be more interesting to mention how your idea developed in phases or how you tried to put it into action."""
        concept = """3)Concept = A concept is perceived in your mind, and it serves as the blueprint of a more structured qualitative or physical outcome. An engaging concept becomes the backbone of your intellectual development or innovations you create in the physical world. As you explain your concept, try to share why the concept is meaningful to you as a person, scholar, or change maker for society. If you have a mentor, you can elaborate on how they have guided and influenced your growth."""
    
        topics_result = "Key Topics :" + topic + idea + concept
        
        # Select topic
        topics = {}
        topics['topic'] = "topic"
        topics['idea'] = "idea"
        topics['concept'] = "concept"
        
        selected_topic = selectFromDict(topics, 'topic')
        
        if selected_topic == 'topic':
            desc = topic
        elif selected_topic == 'idea':
            desc = idea
        else: #concept
            desc = concept
        
    else: # selected_prompt_input == """Share an essay on any topic of your choice. It can be one you've already written, one that responds to a different prompt, or one of your own design.""":
        openTopic = """1)Open topic : You can choose to write about any topic of your choice for this prompt. Because prompts 1-6 cover a wide range of topics, please make sure your topic doesn't fall under the previous prompts. While you can talk about anything, you should ensure that your essay contains the common element sought by all the prompts above and sufficiently explains your realization or lesson. The essential element in a personal statement is showing your personal growth, so you should describe how your topic led you to develop a more mature view of the world."""
       
        topics_result = "2)Key Topic :" + openTopic
        
        # Select topic
        topics = {}
        topics['openTopic'] = "openTopic"
        
        selected_topic = selectFromDict(topics, 'topic')
        
        desc = openTopic
        
        # desc : 선택하 주제에 대한 설명 
    return selected_prompt_input,topics_result, selected_topic, desc


In [18]:
#Step_02()

In [19]:
# introduction
def Step_03(Selected_topic_input):
    print("")
    print("Step #3")
    print("You choese to write about ", Selected_topic_input)
    print("How does your story begin?")
    print("Where and when?")
    print("Who are in it?")
    print("What happens in the beginning?")
    print("Your response here. Minimum 50 words.")
    
    try:
        intro_input = input()
    except:
        intro_input = input()
        ExceptionInput(intro_input)
        Step_03(Selected_topic_input)
        
        
    #intro_input = input()
    print("processing...")
    
    return intro_input, Selected_topic_input

In [20]:
#Summarization
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

In [21]:
def summarization(input_txt):
    preprocess_text = input_txt.strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    #print ("original text preprocessed: \n", preprocess_text)

    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


    # summmarize 
    summary_ids = model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=50, # Msx 2 senteces
                                        early_stopping=True)

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return output

In [22]:
# test summarization
# input_txt_test = """I add the critically measured sugary tea mixture to the gallon jar containing the slimy, white, disc-shaped layers of the symbiotic culture of bacteria and yeast.
# Now to wait.  After exactly seven days, I pour the liquid into a fermentation-grade glass bottle with a ratio of 20% pomegranate juice and 80% fermented tea. I place it on my kitchen counter, periodically checking it to relieve the built-up CO2.
# Finally, after an additional seventy-two hours, the time comes to try it. I crack the seal on the bottle, leaning over to smell what I assume will be a tangy, fruity, delicious pomegranate solution. and it smells like rotten eggs. The insufferable stench fills my nostrils and crushes my confidence. I'm momentarily taken aback, unable to understand how I went wrong when I followed the recipe perfectly. """

# summarized_txt = summarization(input_txt_test)

# print(summarized_txt)

In [23]:
# Rising Action
# input_txt : 학생이 작성한 Original Text
def Step_04(input_txt, selected_topic):
    print("")
    print("Step #4")
    print("What happens next? AI's ideas to get you inspired!")
    gen_sentence = GenText_RisingAction_for_Step04(input_txt)
    print("")
    print(gen_sentence)
    print("")
    print("Inspired? Let's work on the rising action. What happens after the intro? -Any conflicts, problems, or questions you've dealt with?")
    print("Your response here. Minimum 50 words.")
    
    try:
        user_response = input()
    except:
        user_response = input()
        ExceptionInput(user_response)
        Step_04(input_txt)
    
    # user_response = input()
    
    #gen_sentence : 생성된 rising aciton  6개의 문장들
    #user_response : 추가입력하는 사용자  rising action 입력문장
    return gen_sentence, user_response, selected_topic

In [24]:
# intro_ori_input_sent : intro 입력 원본문장
# intro_input_summarised_txt: 이미 요약된 문장으로 intro 부분
# risingAction_input_txt: Rising action 입력문장
def Step_05(intro_ori_input_sent, intro_input_summarised_txt, risingAction_input_txt):
    print("")
    print("Step #5")
    print("Your story so far.")
    risingAciton_summary_re = summarization(risingAction_input_txt)
    # Intro + Rising Action 내용 3-4문장 간추림
    print(intro_input_summarised_txt + risingAciton_summary_re)
    pre_sum_climax_sentences = intro_input_summarised_txt + risingAciton_summary_re
    
    print("")
    #Generate 되는 문장들은 학생의 original writing들 기반 (summary 아니고)
    gen_climax_sentence = GenText_Climax_for_Step05(intro_ori_input_sent + risingAction_input_txt)
    print(gen_climax_sentence)
    
    print("What happens next? AI's ideas to get you inspired!")
    print("")
    print("Describe the most dramatic moment in the story. What happens at the story's peak? How did your conflict, problem, or question resolve?")
    print("Your response here. Minimum 50 words.")
    
    try:
        climax_input = input()
    except:
        climax_input = input()
        ExceptionInput(climax_input)
        Step_05(intro_ori_input_sent, intro_input_summarised_txt, risingAction_input_txt)
    
    # climax_input = input()
    print("Your Climax input:", climax_input)
    
    summarised_climax = summarization(climax_input)
    
    # risingAciton_summary_re : summary rising action
    # gen_climax_sentence : 생성된 Climax idea sentences
    # climax_input : Use input climax
    # summarised_climax : summary climax
    return risingAciton_summary_re, gen_climax_sentence, climax_input, summarised_climax
    

In [25]:
# intro_ori_input_sent : intro 입력 원본문장
# intro_input_summarised_txt: 이미 요약된 문장으로 intro 부분
# risingAction_input_txt: Rising action 입력문장

# add additional sentences.
def Step_05_Climax(intro_ori_input_sent, intro_input_summarised_txt, risingAction_input_txt, selected_topic):
    print("")
    print("Step #5")
    print("Your story so far.")
    risingAciton_summary_re = summarization(risingAction_input_txt)
    # Intro + Rising Action 내용 3-4문장 간추림
    print(intro_input_summarised_txt + risingAciton_summary_re)
    pre_sum_climax_sentences = intro_input_summarised_txt + risingAciton_summary_re
    
    print("")
    #Generate 되는 문장들은 학생의 original writing들 기반 (summary 아니고)
    intro_ori_input_sent_ = str(intro_ori_input_sent)
    risingAction_input_txt_ = str(risingAction_input_txt)
    gen_climax_sentence = GenText(intro_ori_input_sent_ + risingAction_input_txt_)
    print(gen_climax_sentence)
    
    print("What happens next? AI's ideas to get you inspired!")
    print("")
    print("Describe the most dramatic moment in the story. What happens at the story's peak? How did your conflict, problem, or question resolve?")
    print("Your response here. Minimum 50 words.")
    
    try:
        climax_input = input()
    except:
        climax_input = input()
        ExceptionInput(climax_input)
        Step_05(intro_ori_input_sent, intro_input_summarised_txt, risingAction_input_txt)
    
    # climax_input = input()
    print("Your Climax input:", climax_input)
    
    summarised_climax = summarization(climax_input)
    
    # risingAciton_summary_re : summary rising action
    # gen_climax_sentence : 생성된 Climax idea sentences
    # climax_input : Use input climax
    # summarised_climax : summary climax
    return risingAciton_summary_re, gen_climax_sentence, climax_input, summarised_climax, selected_topic

In [26]:
def Step_06(sum_intro, sum_rasingAciton, sum_climax, intro_ori_input_sent, risingAction_input_txt, climax_input_txt, selected_topic):
    print("")
    print("Step #6")
    print("How did your conflict, problem, or question resolve?")
    # show summary of Intro + Rasing Action + Climax
    step_6_que_re = sum_intro + sum_rasingAciton + sum_climax
    #print(step_6_que_re)
    print("")
    print("What happens next? AI's ideas to get you inspired!")
    
    intro_ori_input_sent_ = str(intro_ori_input_sent)
    risingAction_input_txt_ = str(risingAction_input_txt)
    climax_input_txt_ = str(climax_input_txt)
    
    gen_resolution = GenText_Resolution_for_Step06(intro_ori_input_sent_ + risingAction_input_txt_ + climax_input_txt_, selected_topic)
    print("Generated resolution sentences :", gen_resolution)
    print("")
    print("""Describe how your story ends. What happens at the end of your story? -How did this experience change your perspective?""")
    print("Your response here. Minimum 50 words.")
    
    try:
        Resolution_input = input()
    except:
        Resolution_input = input()
        ExceptionInput(Resolution_input)
        Step_06(sum_intro, sum_rasingAciton, sum_climax, intro_ori_input_sent, risingAction_input_txt, climax_input_txt)
    
    # Resolution_input = input()
    print("Your resolution input :", Resolution_input)
    
    
    intro_ori_input_sent_re = str(intro_ori_input_sent_)
    risingAction_input_txt_re = str(risingAction_input_txt_)
    climax_input_txt_re = str(climax_input_txt_)
    gen_resolution_re = str(gen_resolution)
    genFullSentences = intro_ori_input_sent_re + risingAction_input_txt_re + climax_input_txt_re + gen_resolution_re
    # gen_resolution : 생성한 Resolution sentences
    # Resolution_input : User input resolution
    # genFullSentences : 생성한 모든 문장 -> 이것은 감성분석에 입력될거임
    return gen_resolution, Resolution_input, genFullSentences

In [27]:
# For test
# doc = """ Supervised learning is the machine learning task of learning a function that maps an input to an output basedon example input-output pairs.[1] It infers a function from labeled training data consisting of a set of training examples.[2] In supervised learning, each example is a pair consisting of an input object (typically a vector) and a desired output value (also called the supervisory signal). A supervised learning  algorithm analyzes the training data and produces an  inferred function, which can be used for mapping new examples. An optimal scenario will allow for the algorithm to correctly determine the class labels for unseen instances. This requires the learning algorithm to  generalize from the training data to unseen situations in a 'reasonable' way (see inductive bias)."""


In [28]:
### Step_07 
### Extractinon Key Topics

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Maximal Marginal Relevance 적용
# 문서와 가장 유사한 키워드/키프레이즈를 선택하는 것으로 시작합니다. 그런 다음 문서와 유사하고 이미 선택한 키워드/키프레이즈와 유사하지 않은 새 후보를 반복적으로 선택

def extractKeywords(doc):

    n_gram_range = (1, 1)
    stop_words = "english"


    # Extract candidate words/phrases
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
    candidates = count.get_feature_names()



    #model = SentenceTransformer('all-MiniLM-L6-v2')
    model = SentenceTransformer('all-distilroberta-v1')
    doc_embedding = model.encode([doc])
    candidate_embeddings = model.encode(candidates)

    diversity = 0.4

    top_n = 5
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
    word_similarity = cosine_similarity(candidate_embeddings)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(candidates)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [candidates[idx] for idx in keywords_idx]



In [29]:
# result = extractKeywords(doc)

# print(result)

In [30]:
# Key Emotions : part 01

import torch.nn as nn
from transformers import BertPreTrainedModel, BertModel


class BertForMultiLabelClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, self.config.num_labels)
        self.loss_fct = nn.BCEWithLogitsLoss()

        self.init_weights()

    def forward(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            labels=None,
    ):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            loss = self.loss_fct(logits, labels)
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [39]:
# Key Emotions : part 02

from transformers import BertTokenizer

import torch
import pandas as pd
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
import numpy as np

tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-ekman")
model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-ekman")

#데이터 전처리 
def cleaning(datas):

    fin_datas = []

    for data in datas:
        # 영문자 이외 문자는 공백으로 변환
        only_english = re.sub('[^a-zA-Z]', ' ', data)
    
        # 데이터를 리스트에 추가 
        fin_datas.append(only_english)

    return fin_datas


def get_ai_emotion(input_text) :

    joy = 0
    anger = 0
    sadness = 0
    surprise = 0
    fear = 0
    neutral = 0
    
    list_max = []
    
    try : 
        re_text = input_text.split(".")
        texts = cleaning(re_text)
        results = []
        for txt in texts:
            inputs = tokenizer(txt,return_tensors="pt")
            outputs = model(**inputs)
            scores =  1 / (1 + torch.exp(-outputs[0]))  # Sigmoid
            threshold = .3
            for item in scores:
                labels = []
                scores = []
                for idx, s in enumerate(item):
                    if s > threshold:
                        labels.append(model.config.id2label[idx])
                        scores.append(s)
                results.append({"labels": labels, "scores": scores})

        print("results :", results)

        re_emot = results
        
        df = pd.DataFrame(re_emot)

        #결과물을 다시 감정유형별 비율로 계산하여 새로운 데이터프레임을 만들자!
        # result_emotion = df['labels'].value_counts(normalize=True, sort=True, ascending=False, dropna=True) #문장 전체에서 각 값의 상대적 비율을 게산
        result_emotion = df['labels'].value_counts(normalize=True) #문장 전체에서 각 값의 상대적 비율을 게산
        #print("index", result_emotion.index[0][0])

        result_emotion_list = result_emotion.tolist()
              
        idx=0
        
        for idx in range(len(result_emotion)) : 
                
                
                if  'joy' in result_emotion.index[idx][0]: 
                    joy += round(float(result_emotion_list[idx]),2)
                    print("joy >>>>>>>>>>>>>>>>:", joy)
                    
                    
                elif 'anger' in result_emotion.index[idx][0]:
                    anger += round(float(result_emotion_list[idx]),2)
                    print("anger >>>>>>>>>>>>>>>>:", anger)
                    
                    

                elif 'sadness' in  result_emotion.index[idx][0]:
                    sadness += round(float(result_emotion_list[idx]),2)
                    print("sadness >>>>>>>>>>>>>>>>:", sadness)

                    

                elif 'surprise' in result_emotion.index[idx][0]:
                    surprise += round(float(result_emotion_list[idx]),2)
                    print("surprise >>>>>>>>>>>>>>>>:", surprise)

                    

                elif 'fear' in result_emotion.index[idx][0]:
                    fear += round(float(result_emotion_list[idx]),2)
                    print("fear >>>>>>>>>>>>>>>>:", fear)

                    

                elif 'neutral' in result_emotion.index[idx][0]:
                    neutral += round(float(result_emotion_list[idx]),2)
                    print("neutral >>>>>>>>>>>>>>>>:", neutral)

                idx = idx + 1

            
        list_max = [joy,anger,sadness,surprise,fear,neutral]

        print("max:",list_max.index(max(list_max))) 
        
        
    except Exception as e : 
        print ("------------------------->error:",e)
    
    try : 
        max_value = list_max.index(max(list_max))+1 
    
    except Exception as e :
        max_value = 2 
        print("Exception_-------->",e)
            
            
            
    data = {
        "joy" : round(joy, 3),
        "anger" : round(anger, 3),
        "sadness" : round(sadness, 3),
        "surprise" : round(surprise, 3),
        "fear" : round(fear, 3),
        "neutral" : round(neutral, 3),
        #"max": max_value
    }
    
    data_re = sorted(data.items(), key=lambda x: x[1], reverse=True)

    
    return data_re 


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kimkwangil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
text = """ Tom was in love with Cyndi. He really loved her dearly. She loved Tom too because he was so nice and kind. They would walk in the peaceful park gazing at the bright pink cherry blossom that colored the crisp spring air. Then they got into a big fight. Tom began to hate Cyndi's guts. Cyndi cheated on Tom and she began to ignore Tom. He eventually became furious. Back at their house, Tom began to slap Cyndi and she was crying miserably. However, the sun shined upon them when they won the lotto. Both of them were ecstatic and overjoyed with happiness. They where so happy that they jumped around in celebration. Nonetheless, the happiness never lasted too long. Cyndi began to cheat on Tom again and they went bankrupt very quickly. Fights and quarrels continued in the old torn smelly apartment. It really sucked for both of them. Extreme sadness came over them. But then Cyndi bought another lotto and she won again. This time, it was even more money and they were extremely excited. To celebrate the second winning, then went to a casino and began gambling. They were overly excited because they won money at the fancy casino too. Nevertheless, Tom had too much drink and died. Cyndi was so sad. Didn't know what to do and weeped in despair. """
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",get_ai_emotion(text))


results : [{'labels': ['joy'], 'scores': [tensor(0.9299, grad_fn=<UnbindBackward0>)]}, {'labels': ['joy'], 'scores': [tensor(0.9519, grad_fn=<UnbindBackward0>)]}, {'labels': ['joy'], 'scores': [tensor(0.9759, grad_fn=<UnbindBackward0>)]}, {'labels': ['joy', 'neutral'], 'scores': [tensor(0.5449, grad_fn=<UnbindBackward0>), tensor(0.5627, grad_fn=<UnbindBackward0>)]}, {'labels': ['neutral'], 'scores': [tensor(0.7841, grad_fn=<UnbindBackward0>)]}, {'labels': ['anger', 'neutral'], 'scores': [tensor(0.5324, grad_fn=<UnbindBackward0>), tensor(0.4075, grad_fn=<UnbindBackward0>)]}, {'labels': ['neutral'], 'scores': [tensor(0.6375, grad_fn=<UnbindBackward0>)]}, {'labels': ['neutral'], 'scores': [tensor(0.6082, grad_fn=<UnbindBackward0>)]}, {'labels': ['neutral'], 'scores': [tensor(0.5659, grad_fn=<UnbindBackward0>)]}, {'labels': ['neutral'], 'scores': [tensor(0.8980, grad_fn=<UnbindBackward0>)]}, {'labels': ['joy'], 'scores': [tensor(0.9537, grad_fn=<UnbindBackward0>)]}, {'labels': ['joy'], 'sc

In [33]:
# Story Organized by AI

import nltk
import json
from collections import OrderedDict

nltk.download('punkt')

from nltk.tokenize import sent_tokenize
from torch import relu

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("hetpandya/t5-small-tapaco")
model = AutoModelForSeq2SeqLM.from_pretrained("hetpandya/t5-small-tapaco")

def ExceptionInput(input_txt):
    if (input_txt.isdigit()):
        print("An incorrect input was detected. Enter your essay and try again.")
        StartEdit()

    elif (len(input_txt) <= 20):
        print("The sentence is too short. Enter your essay and try again.")
        StartEdit()

    elif (input_txt == None):
        print("Enter the essay you want to paraphrase.")
        StartEdit()


    return input_txt


# create a function for the paraphrase
def paraphrase(sentence):
   
  sentence = "paraphrase: " + sentence + " </s>"
  encoding = tokenizer.encode_plus(sentence,padding=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]
   
  outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    do_sample=True, #샘플링 전략 사용
    max_length=512, # 최대 디코딩 길이는 50
    top_k=5, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
    top_p=0.75, # 누적 확률이 98%인 후보집합에서만 생성
    num_return_sequences=5) #3개의 결과를 디코딩해낸다

  output = tokenizer.decode(outputs[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)
 
  return(output)


def StoryOrgByAI(intro, risingAction, climax, resolution):
    intro_re = " ".join([paraphrase(sent) for sent in sent_tokenize(intro)])
    risingAction_re = " ".join([paraphrase(sent) for sent in sent_tokenize(risingAction)])
    climax_re = " ".join([paraphrase(sent) for sent in sent_tokenize(climax)])
    resolution_re = " ".join([paraphrase(sent) for sent in sent_tokenize(resolution)])
    
    resultData = OrderedDict()
    resultData["intro"] = intro_re
    resultData["risingAction"] = risingAction_re
    resultData["climax"] = climax_re
    resultData["resolution"] = resolution_re
    
    return resultData
        

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kimkwangil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
# intro = """Volunteering at a cancer treatment center has helped me discover my path"""
# risingAction = """When I see patients trapped in not only the hospital but also a moment in time by their diseases, I talk to them. For six hours a day, three times a week, Ivana is surrounded by IV stands, empty walls, and busy nurses that quietly yet constantly remind her of her breast cancer. """
# climax = """Her face is pale and tired, yet kind--not unlike my grandmother’s. I need only to smile and say hello to see her brighten up as life returns to her face. Upon our first meeting, she opened up about her two sons, her hometown, and her knitting group--no mention of her disease."""
# resolution = """Through my work, I can accept the shovel without burying my grandmother’s memory."""

# StoryOrgByAI(intro, risingAction, climax, resolution)


In [37]:
def Run():
    Step_01()
    
    print("")
    
    selected_topic = Step_02()
    
    print("")
    
    # 선택한 토픽에 대한 설명
    print(selected_topic[3])
    
    # intro_input_txt:  학생이 작성한 intro sentence
    # result of intro_input_txt => intro_input, Selected_topic_input
    intro_input_txt = Step_03(selected_topic[2])
    
    print("")
    
    #intro summary sentence
    intro_summarized_txt = summarization(intro_input_txt[0])
    
    print("")
    
    print('summary intro :', intro_summarized_txt)
    
    print("")

    
    in_selected_topic = str(intro_input_txt[1])
    result = Step_04(intro_input_txt, in_selected_topic)
    #gen_sentence  > result[0]: 생성된 6개의 risng action 문장들
    #user_response  > result[1]: 사용자 rising action입력문장
    #selected_topic > result[2] 
    
    print("")
    
    step_5_re = Step_05_Climax(intro_input_txt, intro_summarized_txt, result[1], result[2])
    print("Generated 6 sentences of the climax idea", step_5_re[1]) # 생성된 Climax idea sentences
    print("")
    print("User climax input sentence :", step_5_re[2]) # User climax input
    print("")
    
    sum_intro = intro_summarized_txt
    sum_rasingAciton = str(step_5_re[0])
    sum_climax = str(step_5_re[2]) 
    intro_ori_input_sent = intro_input_txt
    risingAction_input_txt = str(result[1])
    climax_input_txt = str(step_5_re[2])
    
    selected_topic = step_5_re[4]
    
    gen_resolution_re = Step_06(sum_intro, sum_rasingAciton, sum_climax, intro_ori_input_sent, risingAction_input_txt, climax_input_txt, selected_topic)
    print("")
    # gen_resolution_re[0] : 생성한 Resolution sentences
    # gen_resolution_re[1] : User input resolution
    # gen_resolution_re[2] : 생성한 모든 문장(intro, rasingAction, climax, resolution)
    print("Generated the resolution sentences :", gen_resolution_re[0])
    print("")
    print("User input resolution sentence :", gen_resolution_re[1])
    print("")
    
    # Key Topics
    doc = str(gen_resolution_re[2])
    keyTopicsRe = extractKeywords(doc)
    

    # Story Organized by AI
    intro = str(intro_input_txt)
    print("intro : ", intro)
    risingAction = str(result[1])
    print("risingAction :", risingAction)
    climax = str(step_5_re[2])
    print("climax :", climax)
    resolution = str(gen_resolution_re[1])
    print("resolution :", resolution)
    
    story_ai_input = intro + risingAction + climax + resolution
    
    # Story Organized by User(human)
    intro_ori_input_sent = str(intro_input_txt)
    risingAction_input_txt = str(result[1])
    climax_input_txt = str(step_5_re[2])
    user_resolution = str(gen_resolution_re[1])
    
    story_user_input = intro_ori_input_sent + risingAction_input_txt + climax_input_txt + user_resolution
    
    # Key Emotions
    emotions_re = get_ai_emotion(story_user_input)
    
    print(" ======+++++++======= ")
    
    story_re = StoryOrgByAI(intro, risingAction, climax, resolution)
    
    
    # gen_resolution_re[0] : 생성한 Resolution sentences
    # gen_resolution_re[1] : User input resolution
    # keyTopicsRe : Key Topics
    # emotions_re : 감성분석
    # story_re : Para-phrasing 문장들 intro, rising action, climax, resolution
    
    data = {
        "ViewYourStory" : selected_topic[0],
        "YouChoseToWriteAbout" : selected_topic[2],
        "KeyTopics" : keyTopicsRe,
        "KeyEmotions" : emotions_re,
        # StoryOrganizedByAI
        "Intro" : story_re["intro"] ,
        "RisingAction" : story_re["risingAction"],
        "Climax" : story_re["climax"],
        "Resolution" : story_re["resolution"]
    }
    
#     return gen_resolution_re[0], gen_resolution_re[1], keyTopicsRe, emotions_re, story_re
    return data


In [38]:
Run()


Step #1
I am using Common App OR Coalition App
Select a Question:
1) CommonApp
2) CoalitionApp
Question: 1
Selected Question: CommonApp
selected_app_input[0] : C

Step #2
Select prompt
Select a question:
1) prompt01
2) prompt02
3) prompt03
4) prompt04
5) prompt05
6) prompt06
7) prompt07
question: 6
Selected question: Describe a topic, idea, or concept you find so engaging that it makes you lose all track of time. Why does it captivate you? What or who do you turn to when you want to learn more?
Select a topic:
1) topic
2) idea
3) concept
topic: 3
Selected topic: concept

3)Concept = A concept is perceived in your mind, and it serves as the blueprint of a more structured qualitative or physical outcome. An engaging concept becomes the backbone of your intellectual development or innovations you create in the physical world. As you explain your concept, try to share why the concept is meaningful to you as a person, scholar, or change maker for society. If you have a mentor, you can elabo

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



summary intro : I refused to let go of my grandmother, to accept a death I hadn't seen coming, and to believe that an illness could not only interrupt, but steal my beloved life.


Step #4
What happens next? AI's ideas to get you inspired!
-------
input text: ('They covered the precious mahogany coffin with a brown amalgam of rocks, decomposed organisms, and weeds. It was my turn to take the shovel, but I felt too ashamed to dutifully send her off when I had not properly said goodbye. I refused to throw dirt on her. I refused to let go of my grandmother, to accept a death I had not seen coming, to believe that an illness could not only interrupt, but steal a beloved life.', 'concept')Then, an unexpected thing happened.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated sentence :  This time her end came naturally, as if by following the story and then my end would be much easier; or perhaps, it was that death was only a moment away. Then, she died at exactly the right time, her last sentence about having to remember her daughter, that her whole life was'my life', giving her just the grace, the freedom of a miracle. And now, I could touch her, smell her skin. TextToTextResult(text='The words in her notebook were all there, in the sound.')
-------
input text: ('They covered the precious mahogany coffin with a brown amalgam of rocks, decomposed organisms, and weeds. It was my turn to take the shovel, but I felt too ashamed to dutifully send her off when I had not properly said goodbye. I refused to throw dirt on her. I refused to let go of my grandmother, to accept a death I had not seen coming, to believe that an illness could not only interrupt, but steal a beloved life.', 'concept')Then, a sad thing happened.


...

Generated sentence :  The concept seems to have been based on a belief that there is a straight line between the past, the present and the future. With this is mind, we are tempted to think along the same lines when considering the life of a departed grandmother or of our student with poor grades. In each case, however, it is different.In this lifetime, one of the lessons I have learned is that the concept we have in mind to be taught with is not the correct concept. TextToTextResult(text='When we look at life, we look at life.')
Generated sentence :  The concept seems to have been based on a belief that there is a straight line between the past, the present and the future. With this is mind, we are tempted to think along the same lines when considering the life of a departed grandmother or of our student with poor grades. In each case, however, it is different.In this lifetime, one of the lessons I have learned is that the concept we have in mind to be taught with is not the correct c

However, a simple walk on a hiking trail behind my house made me open my own eyes to the truth. Over the years, everything--even honoring my grandmother--had become second to school and grades. As my shoes humbly tapped against the Earth, the towering trees blackened by the forest fire a few years ago, the faintly colorful pebbles embedded in the sidewalk, and the wispy white clouds hanging in the sky reminded me of my small though nonetheless significant part in a larger whole that is humankind and this Earth. Before I could resolve my guilt, I had to broaden my perspective of the world as well as my responsibilities to my fellow humans.   
Your resolution input : However, a simple walk on a hiking trail behind my house made me open my own eyes to the truth. Over the years, everything--even honoring my grandmother--had become second to school and grades. As my shoes humbly tapped against the Earth, the towering trees blackened by the forest fire a few years ago, the faintly colorful p

/Users/kimkwangil/opt/anaconda3/envs/py37NLP/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
Batches: 100%|██████████| 9/9 [00:01<00:00,  7.17it/s]


intro :  ('They covered the precious mahogany coffin with a brown amalgam of rocks, decomposed organisms, and weeds. It was my turn to take the shovel, but I felt too ashamed to dutifully send her off when I had not properly said goodbye. I refused to throw dirt on her. I refused to let go of my grandmother, to accept a death I had not seen coming, to believe that an illness could not only interrupt, but steal a beloved life.', 'concept')
risingAction : When my parents finally revealed to me that my grandmother had been battling liver cancer, I was twelve and I was angry--mostly with myself. They had wanted to protect me--only six years old at the time--from the complex and morose concept of death. However, when the end inevitably arrived, I wasn’t trying to comprehend what dying was; I was trying to understand how I had been able to abandon my sick grandmother in favor of playing with friends and watching TV. Hurt that my parents had deceived me and resentful of my own oblivion, I com

{'ViewYourStory': 'c',
 'YouChoseToWriteAbout': 'n',
 'KeyTopics': ['mortality', 'past', 'text', 'comprehend', 'learning'],
 'KeyEmotions': [('joy', 0),
  ('anger', 0),
  ('sadness', 0),
  ('surprise', 0),
  ('fear', 0),
  ('neutral', 0)],
 'Intro': 'They covered the precious mahogany coffin with a brown amalgam of rocks, decomposed organisms and weeds. I was unable to take the shovel, but I felt too ashamed to send her off, but I didn\'t say goodbye. I refused to throw dirt on him. I refused to let go of my grandmother, to accept that death that I hadn\'t seen coming, to believe that an illness can not only interrupt, but steal a beloved life. ", "concept""',
 'RisingAction': "When my parents finally revealed to me my grandmother had liver cancer, I was twelve and I was angry - most of all. They wanted to protect me from the complex and morose concept of death. Even when the end inevitably came, I wasn't trying to comprehend what was dying; I tried to understand how I could abandon my